In [0]:
# !pip install torch torchvision

In [0]:
# !python -m pip install torch

In [0]:
# Dependencies
import pandas as pd 
import numpy as np

from time import time
# import torch.nn as nn
# import torch
from sklearn.metrics import classification_report


import warnings
warnings.filterwarnings("ignore")

In [0]:
# Read in data from S3 Buckets
url = "https://job-postings-dataviz.s3.amazonaws.com/fake_jobs_clean.csv"
df = pd.read_csv(url, sep=",",  encoding = "UTF-8" )

# Show DataFrame
# df.head()

In [37]:
# Focus only on the job description. Create a dataFrame with label "fraudulent" and "description"
descrip_df =df [['fraudulent','description']]
descrip_df.head(1)


,fraudulent,description
0,0,"Food52, a fast-growing, James Beard Award-winn..."


In [38]:
# Drop NaN values but dropping duplicates is qustionable
# these duplicates are real data and contribute to both classes
print(len(descrip_df))
descrip_df= descrip_df.dropna()
print(len(descrip_df))
# descrip_df = descrip_df.drop_duplicates()
# print(descrip_df.count())

17880
17879


In [31]:
# Make sure number of real vs fraudulent are more balanced
df_fake= descrip_df[descrip_df['fraudulent'] == 1] 
df_real = descrip_df[descrip_df['fraudulent'] == 0] 
# Count how many Real vs Fraudulent postings
n_f=len(df_fake)
n_n=len(df_real)
print(n_f)
print(n_n)


865
17014


Create a data set with same instances of fake and real : UNDERSTIMATING the real posts

In [32]:
df_new= df_real.sample  ( n_f  , random_state=580)
df_unders = df_new.append(df_fake)
print(len(df_new))
print(len(df_unders))

865
1730


In [0]:
# Shuffle the UNDERSTIMATE dataframe so fraudulent postings more evenly distributed 
df_underst = df_unders.sample(len(df_unders), random_state=580) 
# df_underst.head(20)
X_u = df_underst.iloc[:,1].values
y_u = df_underst.iloc[:,0].values



In [0]:
# Use all data set with both models
X= descrip_df.iloc[:,1].values
y=descrip_df.iloc[:,0].values

### Feature Transformations (Term Frequency times inverse document frequency)


In [40]:
# feature extraction using "bag of words" model. Create feature vectors. 
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))


count_vectorizer = CountVectorizer( stop_words=stopwords.words('english'))
tfidfconverter = TfidfTransformer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
#  apply the transformatins
X_c = count_vectorizer.fit_transform(X)
X_tfidf = tfidfconverter.fit_transform(X_c).toarray()

# y.shape

Create Deep Learning Model

In [0]:
# Break the data in trainand test datasets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y_u , random_state=580)

scale the data before any neural net

In [0]:
# Create a StandardScater model and fit it to the training data
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.utils import to_categorical

# Transform the training and testing data using the X_scaler

X_scaler = StandardScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [0]:
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)


In [0]:

# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

Deep Learning Model 

In [0]:
# first, create a normal neural network with 2 inputs, 6 hidden nodes, and 2 outputs
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

deep_model = Sequential()
deep_model.add(Dense(units=100, activation='relu', input_dim=40))
deep_model.add(Dense(units=100, activation='relu'))
deep_model.add(Dense(units=2, activation='softmax'))

In [0]:
# Compile and fit the model
deep_model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [0]:
deep_model.summary()

In [0]:
# train the Model

deep_model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=100,
    shuffle=True,
    verbose=2
)

Save the model

In [0]:
# save the model
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# Save to the model to the local drive
from sklearn.externals import joblib

filename = '/content/drive/My Drive/DeepL_all.h5'
joblib.dump(predictor, filename)

In [0]:
# Load the model
# loaded_model= joblib.load(filename)
# predictionsl = loaded_model.predict(X_test)